# ROBUST MULTICLASS CLASSIFIER

In [2]:
import numpy
import pandas
import tensorflow
import keras
import datetime
from matplotlib import pyplot
import seaborn
import itertools
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
import tensorflow.keras.backend as K

%matplotlib inline

/home/higoreller/miniconda3/envs/condaenv/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# DATA PRE-PREPARATION
file_path = '../../../dataset/occurrences.xlsx'

df = pandas.read_excel(file_path)
df = df.drop(["rai", "obm_afeto", "qualificacao"], axis=1).loc[0:600,:] 

#REMOVING NULL VALUES
df.loc[pandas.isnull(df["data"])]
df.loc[pandas.isnull(df["naturezas"])]
df.loc[pandas.isnull(df["bairro_cidade"])]
df.loc[pandas.isnull(df["tr"])]
df.loc[pandas.isnull(df["obm_escala"])]
df.loc[pandas.isnull(df["sexo"])]

df = df.loc[df["bairro_cidade"] != "(null)"]
df = df.loc[df["recurso"] != "(null)"]
df = df.loc[df["tr"] != "(null)"]
df = df.loc[df["obm_escala"] != "(null)"]
df = df.loc[df["sexo"] != "(null)"]

#TRANSFORMING "data" COLUMN INTO NEW COLUMNS "dia" e "periodo". ALSO TRANSFORMING "tr" COLUMN INTO "tempo_resposta" COLUMN

def day_name(timestamp):
    weekdays = ('Segunda-feira', 'Terça-feira', 'Quarta-feira', 'Quinta-feira', 'Sexta-feira', 'Sábado', 'Domingo')
    return weekdays[timestamp.weekday()]

def period_of_day(timestamp):
    period = ("Madrugada", "Matutino", "Vespertino", "Noturno")
    # Madrugada 00:00 às 05:59
    # Matutino 06:00 às 11:59
    # Vespertino 12:00 às 17:59
    # Noturno 18:00 às 23:59
    if 0 <= timestamp.hour < 6:
        return period[0]
    elif 6 <= timestamp.hour < 12:
        return period[1]
    elif 12 <= timestamp.hour < 18:
        return period[2]
    elif 18 <= timestamp.hour < 24:
        return period[3]



def response_time(response_time):
    # Muito rápido 0 a 10 minutos
    # Rápido 10 a 15 minutos
    # Médio 15 a 20 minutos
    # Longo 20 a 30 minutos
    # Muito longo 30 a 45 minutos
    # Extremamente longo > 45 minutos

    response_time_metric = ("Muito rápido", "Rápido", "Médio", "Longo", "Muito longo", "Extremamente longo")

    if type(response_time) is datetime.time:

        total_time_in_minutes = response_time.hour*60 + response_time.minute + response_time.second/60

        if 0 <= total_time_in_minutes <= 10:
            return response_time_metric[0]
        elif 10 < total_time_in_minutes <= 15:
            return response_time_metric[1]
        elif 15 < total_time_in_minutes <= 20:
            return response_time_metric[2]
        elif 20 <  total_time_in_minutes <= 30:
            return response_time_metric[3]
        elif 30 < total_time_in_minutes <= 45:
            return response_time_metric[4]
        elif total_time_in_minutes > 45:
            return response_time_metric[5]
        
#Lembrar de remover os valores que não são datetime.time do df["tr"]
df.loc[:, "dia"] = df["data"].apply(day_name)
df.loc[:, "periodo"] = df["data"].apply(period_of_day)
df.loc[:, "tempo_resposta"] = df["tr"].apply(response_time)

#REMOVING "DATA" AND "TR" COLUMNS

df = df.drop(["data", "tr"], axis=1)

#Removing None values
df = df.dropna()
df = df.mask(df.eq('None')).dropna()
df = df.astype(str)
df

/home/higoreller/miniconda3/envs/condaenv/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Cell F221384 is marked as a date but the serial value -736954.9663425926 is outside the limits for dates. The cell will be treated as an error.
  for idx, row in parser.parse():


,naturezas,bairro_cidade,recurso,obm_escala,sexo,dia,periodo,tempo_resposta
5,RESGATE (3)->EMERGÊNCIA CLÍNICA (304)->ACIDENT...,ZONA RURAL - CATALÃO,UR-181,10º BBM - CATALÃO,MASCULINO,Domingo,Madrugada,Muito rápido
6,RESGATE (3)->ACIDENTE DE TRÂNSITO (301)->CAPOT...,ZONA RURAL - NOVA VENEZA,ABT-26,CAEBM,NÃO-INFORMADO,Domingo,Madrugada,Muito longo
7,RESGATE (3)->ACIDENTE DE TRÂNSITO (301)->CAPOT...,ZONA RURAL - NOVA VENEZA,ASA-74,1º BBM - SETOR CENTRAL,NÃO-INFORMADO,Domingo,Madrugada,Longo
8,RESGATE (3)->ACIDENTE DE TRÂNSITO (301)->CAPOT...,ZONA RURAL - NOVA VENEZA,UR-211,CAEBM,NÃO-INFORMADO,Domingo,Madrugada,Muito longo
9,RESGATE (3)->ACIDENTE DE TRÂNSITO (301)->CAPOT...,ZONA RURAL - NOVA VENEZA,USA-21,BSE,NÃO-INFORMADO,Domingo,Madrugada,Muito longo
...,...,...,...,...,...,...,...,...
591,RESGATE (3)->EMERGÊNCIA CLÍNICA (304)->CONVULS...,MORADA DO MORRO - SENADOR CANEDO,UR-165,14º BBM - SENADOR CANEDO,MASCULINO,Segunda-feira,Matutino,Muito rápido
592,RESGATE (3)->AGRESSÃO (303)->POR ANIMAL (PICAD...,VILA AMÁLIA - RIO VERDE,UR-164,4º BBM - RIO VERDE,MASCULINO,Segunda-feira,Matutino,Médio
595,INCÊNDIO URBANO (6)->EDIFICAÇÃO RESIDENCIAL (6...,VILA SANTA MARIA - JATAÍ,ABT-11,13º BBM - JATAÍ,FEMININO,Segunda-feira,Matutino,Muito rápido
596,RESGATE (3)->EMERGÊNCIA CLÍNICA (304)->OUTRO /...,CONJUNTO RIO CLARO I - JATAÍ,UR-221,13º BBM - JATAÍ,MASCULINO,Segunda-feira,Matutino,Longo


## INPUT PARAMETERS HERE

In [4]:
# COOK PARAMETERS
var_types = {'naturezas' : 'str',
             'bairro_cidade' : 'str',
             'recurso' : 'str',
             'dia': 'str',
             'periodo': 'str',
             'tempo_resposta': 'str',
             'obm_escala': 'str',
             }
class_col = 'tempo_resposta'
my_metric = 'accuracy'
relevant_features = 50
my_patience = 50
min_improvement = 0.01
min_neuron_gain = 0.025
random_seed = 33
n_k_folds = 3
learning_rate = 0.01
hidden_activations = ['linear', 'tanh', 'relu']
output_activation = 'softmax'
rollback_on_no_lower_bound_gain = True

my_optimizers = {'adagrad': tensorflow.keras.optimizers.Adagrad(learning_rate=learning_rate),
                 'rmsprop': tensorflow.keras.optimizers.RMSprop(learning_rate=learning_rate),
                 'adam': tensorflow.keras.optimizers.Adam(learning_rate=learning_rate)}

## CHECK TENSORFLOW GPU

In [5]:
tensorflow.config.list_physical_devices('GPU')

[]

In [6]:
# SCATTER PLOT
#g = seaborn.pairplot(df, hue='tempo_resposta', height=3, diag_kind='kde')
#_ = g.map_lower(seaborn.kdeplot, levels=1, color=".2")

In [7]:
# QUANTIFY REPRESENTATION 
dfn_mi_list = []
dfn_data_list = []
for my_var in var_types.keys():
    if my_var == class_col:
        col_class = 'Y'
    else:
        col_class = 'X'

    if var_types[my_var] == 'float': # NUMERIC DATA
        dfn_mi_list.append((col_class, 'continuous', my_var))
        dfn_data_list.append(df[[my_var]].values)

    elif var_types[my_var] == 'str' and col_class == 'X': # CATEGORICAL DATA
        one_hot = OneHotEncoder(sparse=False)
        var_cat = df[[my_var]].to_numpy()
        one_hot.fit(var_cat)
        df_X = pandas.DataFrame(one_hot.transform(var_cat))
        df_X.columns = one_hot.categories_
        for cat in df_X.columns:
            dfn_mi_list.append((col_class, 'categorical', cat[0]))
            dfn_data_list.append(df_X[[cat]].values)
    
    elif var_types[my_var] == 'str' and col_class == 'Y': # CATEGORICAL DATA
        one_hot = OneHotEncoder(sparse=False)
        df_Y_categorical = df[[class_col]]
        Y_array_categorical = df_Y_categorical.to_numpy()
        one_hot.fit(Y_array_categorical)
        df_Y = pandas.DataFrame(one_hot.transform(Y_array_categorical))
        df_Y.columns = one_hot.categories_
        for cat in df_Y.columns:
            dfn_mi_list.append((col_class, 'categorical', cat[0]))
            dfn_data_list.append(df_Y[[cat]].values)

dfn_mi = pandas.MultiIndex.from_tuples(dfn_mi_list)
dfn = pandas.DataFrame(index=df.index, columns=dfn_mi, data=numpy.concatenate(dfn_data_list, axis=1))
dfn = dfn.sort_index(axis=1)
dfn

X                                                          \
         categorical                                                           
    10ª CIBM - POSSE 10º BBM - CATALÃO 11ª CIBM - URUAÇU 11º BBM - PORANGATU   
5                0.0               1.0               0.0                 0.0   
6                0.0               0.0               0.0                 0.0   
7                0.0               0.0               0.0                 0.0   
8                0.0               0.0               0.0                 0.0   
9                0.0               0.0               0.0                 0.0   
..               ...               ...               ...                 ...   
591              0.0               0.0               0.0                 0.0   
592              0.0               0.0               0.0                 0.0   
595              0.0               0.0               0.0                 0.0   
596              0.0               0.0               0.0                 0.0   
600              0.0               0.0               0.0                 0.0   

                                                          \
                                                           
    12ª CIBM - MORRINHOS 12º BBM - GOIÁS 13º BBM - JATAÍ   
5                    0.0             0.0             0.0   
6                    0.0             0.0             0.0   
7                    0.0             0.0             0.0   
8                    0.0             0.0             0.0   
9                    0.0             0.0             0.0   
..                   ...             ...             ...   
591                  0.0             0.0             0.0   
592                  0.0             0.0             0.0   
595                  0.0             0.0             1.0   
596                  0.0             0.0             1.0   
600                  0.0             0.0             0.0   

                                                                              \
                                                                               
    14ª CIBM - PIRES DO RIO 14º BBM - SENADOR CANEDO 15ª CIBM - QUIRINÓPOLIS   
5                       0.0                      0.0                     0.0   
6                       0.0                      0.0                     0.0   
7                       0.0                      0.0                     0.0   
8                       0.0                      0.0                     0.0   
9                       0.0                      0.0                     0.0   
..                      ...                      ...                     ...   
591                     0.0                      1.0                     0.0   
592                     0.0                      0.0                     0.0   
595                     0.0                      0.0                     0.0   
596                     0.0                      0.0                     0.0   
600                     0.0                      0.0                     0.0   

     ...                                                  \
     ...                                                   
     ... ZONA RURAL - NOVA VENEZA ZONA RURAL - PORANGATU   
5    ...                      0.0                    0.0   
6    ...                      1.0                    0.0   
7    ...                      1.0                    0.0   
8    ...                      1.0                    0.0   
9    ...                      1.0                    0.0   
..   ...                      ...                    ...   
591  ...                      0.0                    0.0   
592  ...                      0.0                    0.0   
595  ...                      0.0                    0.0   
596  ...                      0.0                    0.0   
600  ...                      0.0                    0.0   

                                                                \
                             

## ROBUST COOKING

In [8]:
# SET THE RANDOM SEED
numpy.random.seed(random_seed)
tensorflow.random.set_seed(random_seed)

# PREPARE DATA
X = dfn['X'].to_numpy()
my_vars = list(dfn['X'].columns.get_level_values(1))
Y = dfn['Y'].to_numpy()


# NORMALIZE VARIABLES X
X_scaler = StandardScaler()
X_scaler = X_scaler.fit(X)
X_scaled = X_scaler.transform(X)

# NORMALIZE VARIABLES Y
Y_scaler = StandardScaler()
Y_scaler = Y_scaler.fit(Y)
Y_scaled = Y_scaler.transform(Y)

# GET STRATIFIED FOLDS
sk_folds = StratifiedKFold(n_splits=n_k_folds)
sk_folds.get_n_splits(X_scaled, Y_array_categorical)

# TRAINING LOOP
n_classes = df_Y.columns.shape[0]
hidden_layers = 1

# PATTERN SEARCH
search_vector = numpy.identity(len(hidden_activations), dtype=int)
current_configurations = search_vector

still_searching = True
best_net_name = None
best_net = None
best_config = None
best_metric = 0.0
best_metric_val = 0.0
metric_lower_bound = 0.0
result_list = []

print('COOKING BEST NETWORK STARTED...\n')
while still_searching:
    for net_config in current_configurations:
        print('CONFIG: {}'.format(net_config))
        ix_fold = 1
        for train_index, test_index in sk_folds.split(X_scaled, Y_array_categorical):
            print('FOLD: {}/{}'.format(ix_fold, n_k_folds))

            # GET STRATIFIED DATA
            X_train, X_test = X_scaled[train_index], X_scaled[test_index]
            Y_train, Y_test = Y[train_index], Y[test_index]
            for optimizer in my_optimizers.keys():

                # DECLARE NET
                inputs = keras.Input(shape=(X_train.shape[1], ))
                hidden_layer_list = []
                net_name = ''
                for ix_activation in numpy.arange(len(net_config)):
                    f_activation = hidden_activations[ix_activation]
                    n_neuron = net_config[ix_activation]
                    if n_neuron > 0:
                        temp_hidden_layer = keras.layers.Dense(n_neuron, activation=f_activation)(inputs)
                        hidden_layer_list.append(temp_hidden_layer)
                        net_name += '{}{}'.format(f_activation, n_neuron)
                if len(hidden_layer_list) > 1:
                    hidden = keras.layers.Concatenate(axis=1)(hidden_layer_list)
                else:
                    hidden = hidden_layer_list[0]
                outputs = keras.layers.Dense(n_classes, activation=output_activation)(hidden)
                model_name = '{}_{}_F{}_WRAPPER'.format(net_name.upper(), optimizer.upper(), ix_fold)
                my_net = keras.Model(inputs=inputs, outputs=outputs, name=model_name)

                # COMPILE NET
                metric_name = my_metric.lower()
                val_metric_name = 'val_{}'.format(metric_name)
                my_net.compile(optimizer=my_optimizers[optimizer], loss='mse', metrics=[my_metric])
                es_loss = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', patience=my_patience, restore_best_weights=True)
                es_metric = tensorflow.keras.callbacks.EarlyStopping(monitor=metric_name, mode='max', patience=my_patience, min_delta=min_improvement)
                es_val_metric = tensorflow.keras.callbacks.EarlyStopping(monitor=val_metric_name, mode='max',
                                                                        patience=n_k_folds*my_patience, min_delta=min_improvement)

                # FIT WRAPPER
                with tensorflow.device('/gpu:0'):
                    history = my_net.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=X_train.shape[0], 
                                         epochs=10*my_patience, verbose=0, callbacks=[es_loss, es_metric, es_val_metric])

                # CALCULATE VARIABLE RELEVANCE
                Y_est = my_net.predict(X_scaled)
                relevance_dict = {}
                for feature in my_vars:
                    X_relevance = numpy.copy(X_scaled)
                    X_relevance[:, my_vars == feature] = 0.0
                    Y_est_relevance = my_net.predict(X_relevance)
                    Y_relevance = (Y_est - Y_est_relevance)**2
                    feature_relevance = Y_relevance.sum()/Y_relevance.shape[0]
                    relevance_dict[feature] = feature_relevance
                s_relevance = pandas.Series(relevance_dict)
                s_relevance_norm = s_relevance/s_relevance.max()
                s_relevance_norm = s_relevance_norm.sort_values(ascending=False).head(relevant_features)
                selected_features = list(s_relevance_norm.index.values)
                selected_relevance = list(100.0*s_relevance_norm.values)
                
                
                # FINAL NET
                inputs = keras.Input(shape=(len(selected_features), ))
                hidden_layer_list = []
                net_name = ''
                for ix_activation in numpy.arange(len(net_config)):
                    f_activation = hidden_activations[ix_activation]
                    n_neuron = net_config[ix_activation]
                    if n_neuron > 0:
                        temp_hidden_layer = keras.layers.Dense(n_neuron, activation=f_activation)(inputs)
                        hidden_layer_list.append(temp_hidden_layer)
                        net_name += '{}{}'.format(f_activation, n_neuron)
                if len(hidden_layer_list) > 1:
                    hidden = keras.layers.Concatenate(axis=1)(hidden_layer_list)
                else:
                    hidden = hidden_layer_list[0]
                outputs = keras.layers.Dense(n_classes, activation=output_activation)(hidden)
                model_name = '{}_{}_F{}'.format(net_name.upper(), optimizer.upper(), ix_fold)
                my_net = keras.Model(inputs=inputs, outputs=outputs, name=model_name)

                # COMPILE NET
                metric_name = my_metric.lower()
                val_metric_name = 'val_{}'.format(metric_name)
                my_net.compile(optimizer=my_optimizers[optimizer], loss='mse', metrics=[my_metric])
                es_loss = tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', patience=my_patience, restore_best_weights=True)
                es_metric = tensorflow.keras.callbacks.EarlyStopping(monitor=metric_name, mode='max', patience=my_patience, min_delta=min_improvement)
                es_val_metric = tensorflow.keras.callbacks.EarlyStopping(monitor=val_metric_name, mode='max',
                                                                        patience=n_k_folds*my_patience, min_delta=min_improvement)

                # FIT FINAL
                
                ix_relevant = dfn['X'].columns.get_level_values(1).isin(selected_features)
                with tensorflow.device('/gpu:0'):
                    history = my_net.fit(X_train[:, ix_relevant], Y_train, validation_data=(X_test[:, ix_relevant], Y_test), batch_size=X_train.shape[0],
                                        epochs=10*my_patience, verbose=1, callbacks=[es_loss, es_metric, es_val_metric])

                # UNPACK TRAINING RESULTS
                model_loss = history.history['loss'][-1]
                model_val_loss = history.history['val_loss'][-1]
                metric_val = history.history[metric_name][-1]
                val_metric_val = history.history[val_metric_name][-1]
                n_epochs = len(history.history['val_loss'])

                # CHECK IF BEST NET
                best_net_txt = ''
                is_best = False
                if ((metric_val > best_metric + min_improvement) & (val_metric_val >= best_metric_val)) | ((metric_val >= best_metric) & (val_metric_val > best_metric_val + min_improvement)):
                    best_net_name = model_name
                    best_optimizer = optimizer
                    best_config = net_config
                    best_features = selected_features
                    best_net = my_net
                    best_neurons = n_neuron
                    best_metric = metric_val
                    best_metric_val = val_metric_val
                    best_net_txt = '**Best NET!**'
                    is_best = True

                result_dict = {'model_name': model_name, 'optimizer': optimizer, 'net_config': net_config, 'neurons': net_config.sum(), 'fold': ix_fold, 'features': selected_features, 'relevance': selected_relevance, 'loss': model_loss,
                    'val_loss': model_val_loss, metric_name: metric_val, val_metric_name: val_metric_val, 'epochs': n_epochs, 'is_best': is_best, 'net': my_net}
                result_list.append(result_dict)
                if is_best:
                    print('{}_{} > epochs: {} loss: {:.2f} val_loss: {:.2f} {}: {:.0f}% {}: {:.0f}% {}'.format(model_name,'_'.join(selected_features), n_epochs, model_loss, model_val_loss, metric_name, 100.0*metric_val, val_metric_name, 100.0*val_metric_val, best_net_txt))

                # IF MAXIMIZED METRICS BREAK
                maximized_metrics = (val_metric_val == 1.0) & (metric_val == 1.0)
                if maximized_metrics:
                    break
            
            if maximized_metrics:
                break

            ix_fold += 1

        if maximized_metrics:
            break

    # TEST IF MORE NEURONS ARE NECESSARY
    dont_need_extra_neurons = False
    total_neurons = net_config.sum()

    if total_neurons > 1:
        
        # INCREASING NEURONS DIDNT ACHIEVE BEST NET
        if best_neurons < total_neurons:
            dont_need_extra_neurons = True
            print('NETs w {} neurons didnt improve {}!'.format(total_neurons, metric_name))
            break
    
    # CALCULATE CONFIG LOWER BOUND
    df_results = pandas.DataFrame(result_list)
    ix_config = numpy.array([numpy.linalg.norm(best_config - x) for x in df_results.net_config]) == 0.0
    lower_bound_est = df_results[(df_results.optimizer == best_optimizer) & ix_config][val_metric_name].min()

    # TEST IF LOWER BOUND IMPROVED
    lower_bound_didnt_improve = False
    if total_neurons > 1:
        neuron_gain = 0.0
        if lower_bound_est > metric_lower_bound:
            if metric_lower_bound > 0.0:
                neuron_gain = lower_bound_est/metric_lower_bound - 1.0
                if neuron_gain <= min_neuron_gain:
                    print('NETs w {} neurons didnt improve min {} {:.1f} neuron gain {:.1f} / min {:.1f}!'.format(total_neurons, val_metric_name, 100.0*lower_bound_est, 100.0*neuron_gain, 100.0*min_neuron_gain))
                    lower_bound_didnt_improve = True
                    break
                else:
                    print('NETs Neuron Gain w/ {} neurons: {:.1f} >> {:.1f} = {:.2f}%'.format(total_neurons, 100.0*metric_lower_bound, 100.0*lower_bound_est ,100.0*neuron_gain))
            else:
                print('NET Neural Gain From 0.0 >> {:.1f}%'.format(100.0*lower_bound_est))
        else:
            print('No Lower Bound Gain on {} neurons for {}: {:.1f} >> {:.1f}'.format(total_neurons, best_optimizer, 100.0*metric_lower_bound, 100.0*lower_bound_est))
            lower_bound_didnt_improve = True
            break
    else:
        print('BEST CONFIG {} & min quality for {}: {:.1f}%'.format(best_config, best_optimizer.upper(), 100.0*lower_bound_est))
    
    metric_lower_bound = lower_bound_est

    # ALTER CONFIG FROM BEST CONFIG IF NEEDED
    if maximized_metrics or dont_need_extra_neurons or lower_bound_didnt_improve:
        still_searching = False
    else:
        current_configurations = best_config + search_vector

COOKING BEST NETWORK STARTED...

CONFIG: [1 0 0]
FOLD: 1/3


2022-08-24 16:44:11.031731: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-24 16:44:11.040309: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/500
1/1 [==============================] - 1s 735ms/step - loss: 0.1496 - accuracy: 0.1809 - val_loss: 0.1425 - val_accuracy: 0.2421
Epoch 2/500
1/1 [==============================] - 0s 59ms/step - loss: 0.1496 - accuracy: 0.1809 - val_loss: 0.1425 - val_accuracy: 0.2421
Epoch 3/500
1/1 [==============================] - 0s 142ms/step - loss: 0.1496 - accuracy: 0.1809 - val_loss: 0.1425 - val_accuracy: 0.2421
Epoch 4/500
1/1 [==============================] - 0s 126ms/step - loss: 0.1495 - accuracy: 0.1809 - val_loss: 0.1424 - val_accuracy: 0.2421
Epoch 5/500
1/1 [==============================] - 0s 114ms/step - loss: 0.1495 - accuracy: 0.1809 - val_loss: 0.1424 - val_accuracy: 0.2421
Epoch 6/500
1/1 [==============================] - 0s 89ms/step - loss: 0.1495 - accuracy: 0.1809 - val_loss: 0.1424 - val_accuracy: 0.2421
Epoch 7/500
1/1 [==============================] - 0s 99ms/step - loss: 0.1494 - accuracy: 0.1809 - val_loss: 0.1424 - val_accuracy: 0.2421
Epoch 8/500
1/1 

In [9]:
# ROLL BACK TO LESS NEURON IF DIDNT IMPROVE
if lower_bound_didnt_improve and rollback_on_no_lower_bound_gain:
    best_row = df_results[(df_results.neurons == n_neuron - 1) & df_results.is_best].tail(1)
    best_net_name = best_row['model_name'].values[0]
    best_optimizer = best_row['optimizer'].values[0]
    best_net = best_row['net'].values[0]
    best_config = best_row['net_config'].values[0]
    best_features = best_row['features'].values[0]
    best_neurons = best_row['neurons'].values[0]
    best_metric = best_row[metric_name].values[0]
    best_metric_val = best_row[val_metric_name].values[0]

print('BEST_NET_NAME: {} \n, BEST_OPTIMIZER: {} \n, BEST_NET: {} \n, BEST_CONFIG: {} \n, BEST_FEATURES: {} \n, BEST_NEURONS: {} \n, BEST_METRIC: {} \n, BEST_METRIC_VAL: {} \n,'.format(best_net_name, best_optimizer, best_net, best_config, best_features, best_neurons, best_metric, best_metric_val))

BEST_NET_NAME: LINEAR1TANH1_RMSPROP_F2 
, BEST_OPTIMIZER: rmsprop 
, BEST_NET: <keras.engine.functional.Functional object at 0x7f4aa2dabd30> 
, BEST_CONFIG: [1 1 0] 
, BEST_FEATURES: ['10ª CIBM - POSSE', '10º BBM - CATALÃO', '11ª CIBM - URUAÇU', '11º BBM - PORANGATU', '12ª CIBM - MORRINHOS', '12º BBM - GOIÁS', '13º BBM - JATAÍ', '14ª CIBM - PIRES DO RIO', '14º BBM - SENADOR CANEDO', '15ª CIBM - QUIRINÓPOLIS', '15º BBM - TRINDADE', '16ª CIBM - GOIATUBA', '16º BBM - JARDIM AMÉRICA - INATIVA', '17ª CIBM - ITABERAÍ', '17º BBM - PIRENÓPOLIS', '18º BBM - GOIANÉSIA', '19ª CIBM - SÃO LUÍS M. BELOS', '1ª CIBM - MINAÇU', '1º BBM - SETOR CENTRAL', '1º PBM - ARUANÃ', '2º BBM - SETOR AEROVIÁRIO', '3ª CIBM - SANTA HELENA', '3º BBM - ANÁPOLIS', '4ª CIBM - JARAGUÁ - INATIVA', '4º BBM - RIO VERDE', '5º BBM - LUZIÂNIA', '6ª CIBM - NIQUELÂNDIA', '6º BBM - ITUMBIARA', '7ª CIBM - INHUMAS', '7º BBM - APARECIDA DE GOIÂNIA', '8ª CIBM - CRISTALINA', '8º BBM - PARQUE AMAZÔNIA', '9ª CIBM - PLANALTINA', '9º BBM -